In [1]:
import json
from pprint import pprint
from tqdm import tqdm
# !pip install -U nervaluate
from nervaluate import Evaluator

from exciton.nlp.event_detection import Exciton_ED
from exciton.nlp.event_detection.utils import clean_result

In [2]:
model = Exciton_ED(path_to_model="/tmp/ed_maven_xlmroberta/", device="cuda:0")

In [3]:
def process_spans(input_data):
    input_data["events"] = sorted(input_data["events"], key=lambda x: x["span"][0])
    sents = []
    x = 0
    for sen in input_data["events"]:
        text = input_data["text"][x:sen["span"][0]]
        sents.append({"text": text, "label": "O"})
        text = input_data["text"][sen["span"][0]:sen["span"][1]]
        sents.append({"text": text, "label": sen["label"]})
        x = sen["span"][1]
    text = input_data["text"][x:]
    sents.append({"text": text, "label": "O"})

    tokens = []
    for itm in sents:
        for k, sen in enumerate(itm["text"].split()):
            if itm["label"] == "O":
                tokens.append({"token": sen, "label": itm["label"]})
            else:
                if k == 0:
                    tokens.append({"token": sen, "label": "B-" + itm["label"]})
                else:
                    tokens.append({"token": sen, "label": "I-" + itm["label"]})
    return tokens

In [4]:
data = []
with open("/home/tshi/exciton/datasets/nlp/event_detection/ed_maven_v1/test.jsonl", "r") as fp:
    for line in fp:
        itm = json.loads(line)
        itm["text"] = " ".join(itm["tokens"])
        itm["etokens"] = itm["tokens"]
        data.append(itm)
print(len(data))

gold = []
for itm in tqdm(data):
    results = process_spans(clean_result(itm))
    gold.append([sen["label"] for sen in results])
pred = []
for itm in tqdm(model.predict(data)):
    results = process_spans(itm)
    pred.append([sen["label"] for sen in results])

 31%|████████████████████████████████████████████████████▍                                                                                                                     | 2480/8042 [00:00<00:00, 24799.96it/s]

8042


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8042/8042 [00:00<00:00, 66317.08it/s]


In [5]:
with open("/tmp/ed_maven_xlmroberta/labels.json", "r") as fp:
    labels = json.load(fp)
labels = [wd.replace("B-", "").replace("I-", "") for wd in labels if wd != "O"]
labels = list(set(labels))
evaluator = Evaluator(gold, pred, tags=labels, loader="list")
results, results_by_tag = evaluator.evaluate()
pprint(results)

{'ent_type': {'actual': 16104,
              'correct': 9519,
              'f1': 0.5444717725790768,
              'incorrect': 1859,
              'missed': 7484,
              'partial': 0,
              'possible': 18862,
              'precision': 0.5910953800298062,
              'recall': 0.5046654649559962,
              'spurious': 4726},
 'exact': {'actual': 16104,
           'correct': 11161,
           'f1': 0.6383915803923812,
           'incorrect': 217,
           'missed': 7484,
           'partial': 0,
           'possible': 18862,
           'precision': 0.6930576254346746,
           'recall': 0.5917187997031068,
           'spurious': 4726},
 'partial': {'actual': 16104,
             'correct': 11161,
             'f1': 0.6445976091059887,
             'incorrect': 0,
             'missed': 7484,
             'partial': 217,
             'possible': 18862,
             'precision': 0.6997950819672131,
             'recall': 0.5974711059272612,
             'spurious

In [6]:
text = ["La presencia femenina en el homenaje a los socios del Oviedo: De aquella en el Tartiere todo eran paisanos"]
pprint(model.predict(text))

[{'events': [],
  'text': 'La presencia femenina en el homenaje a los socios del Oviedo: De '
          'aquella en el Tartiere todo eran paisanos'}]


In [7]:
text = ["随着气温回暖，新疆目前已进入春耕春播期，各地通过北斗导航精量播种、干播湿出、种肥分离等技术的运用，助力春耕生产顺利进行。 在新疆昌吉玛纳斯县六户地镇的千亩春小麦种植基地，装有北斗导航系统的大马力拖拉机，带着改良后的新式播种机，正在进行春小麦播种。这种播种机可按照预先设定的线路进行精量播种，而且覆土、铺设滴灌带也同步完成，不仅实现了全程机械化，还提高了作业精度和效率，每天可比传统播种机多播50亩地左右。 "]
pprint(model.predict(text))

[{'events': [{'label': 'Cause_change_of_position_on_a_scale',
              'span': [173, 176],
              'text': '提高了'}],
  'text': '随着气温回暖，新疆目前已进入春耕春播期，各地通过北斗导航精量播种、干播湿出、种肥分离等技术的运用，助力春耕生产顺利进行。 '
          '在新疆昌吉玛纳斯县六户地镇的千亩春小麦种植基地，装有北斗导航系统的大马力拖拉机，带着改良后的新式播种机，正在进行春小麦播种。这种播种机可按照预先设定的线路进行精量播种，而且覆土、铺设滴灌带也同步完成，不仅实现了全程机械化，还提高了作业精度和效率，每天可比传统播种机多播50亩地左右。 '}]


In [8]:
text = ["The snow storm hits new york."]
pprint(model.predict(text))

[{'events': [{'label': 'Attack', 'span': [15, 18], 'text': 'hit'}],
  'text': 'The snow storm hits new york.'}]
